¡Hola!

Mi nombre es Marcos Torres y tengo el gusto de revisar tu código el día de hoy.

Cuando vea algo notable o algún asunto en el notebook, te dejaré un comentario o un hint. Se que encontraras la mejor respuesta para resolver todos los comentarios, de no ser así, no te preocupes en futuras iteraciones dejaré comentarios y pistas más específicos.

Encontrarás comentarios en verde, amarillo o rojo como los siguientes:

<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Buen trabajo. ¡Lo hiciste muy bien!
</div>

<div class="alert alert-block alert-warning">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Nota. Se puede mejorar.
</div>

<div class="alert alert-block alert-danger">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Necesitas corregirlo. Este bloque indica que se requiere una correción. El trabajo no se acepta si tiene estos bloques.
</div>

Puedes responder a mis comentarios usando estos bloques:

<div class="alert alert-block alert-info">
<b>Respuesta del estudiante.</b> <a class="tocSkip"></a>
</div>

# Proyecto: Recomendación de Planes de Megaline

En un esfuerzo por optimizar la experiencia de sus clientes y maximizar su rentabilidad, la compañía móvil **Megaline** busca desarrollar un modelo de machine learning capaz de analizar el comportamiento de sus usuarios y recomendar uno de sus nuevos planes tarifarios: **Smart** o **Ultra**. Este proyecto tiene como objetivo principal identificar patrones en los datos de uso de los clientes que ya se han cambiado a estos planes, permitiendo una clasificación precisa de futuros usuarios según sus hábitos.

## Objetivos del Proyecto
1. **Crear un modelo predictivo** que recomiende el plan tarifario más adecuado para un cliente, basándose en su comportamiento de uso de llamadas, mensajes y datos de internet.
2. **Garantizar una alta exactitud** en las recomendaciones, con un umbral mínimo de exactitud del 75%.
3. **Comparar diferentes modelos y ajustar hiperparámetros**, evaluando su calidad mediante métricas de desempeño.

## Preguntas que se Abordarán
1. ¿Qué tan limpio y adecuado es el dataset para realizar un análisis predictivo?
2. ¿Cómo se puede segmentar de manera efectiva el dataset en conjuntos de entrenamiento, validación y prueba?
3. ¿Cuáles son los modelos más adecuados para esta tarea de clasificación, y cómo impacta el ajuste de hiperparámetros en su desempeño?
4. ¿Qué modelo logra la mayor exactitud en las predicciones y cumple con el umbral establecido?
5. ¿El modelo final es robusto al punto de superar una prueba de cordura, asegurando su capacidad de generalización?

Con este proyecto, **Megaline** espera no solo mejorar la satisfacción del cliente al ofrecerle planes que se ajusten a sus necesidades, sino también optimizar sus operaciones al migrar a más clientes hacia sus nuevos planes tarifarios.


Primero se importa toda la librería, que se va a utilizar durante el proceso. Además de incluir los datos que se van a manipular.

In [25]:
# Importación de librerías necesarias
import pandas as pd  # Para manipulación de datos
import numpy as np # Para operaciones matemáticas
from sklearn.model_selection import train_test_split  # Para dividir los datos
from sklearn.tree import DecisionTreeClassifier  # Modelo de Árbol de Decisión
from sklearn.ensemble import RandomForestClassifier  # Modelo de Bosque Aleatorio
from sklearn.linear_model import LogisticRegression  # Modelo de Regresión Logística
from sklearn.metrics import accuracy_score, classification_report  # Métricas de evaluación

# Cargar los datos
data = pd.read_csv('/datasets/users_behavior.csv')  # Ruta al archivo CSV
    
# Visualizar las primeras filas
data.head()


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Bien, usaste una celda independiente para importar las librerías y otra para leer los datos.
</div>

Ahora se va a proceder con una inspección rápida de la tabla para luego pasar al entrenamiento

In [4]:
# Inspección básica de la estructura del dataset
print("Información general del dataset:")
data.info()

# Verificar si hay valores duplicados
duplicates = data.duplicated().sum()
print(f"\nNúmero de filas duplicadas: {duplicates}")

# Verificar si hay valores nulos
nulls = data.isnull().sum()
print("\nNúmero de valores nulos por columna:")
print(nulls)

# Inspeccionar las estadísticas básicas para detectar valores atípicos o inconsistencias
print("\nEstadísticas básicas del dataset:")
print(data.describe())


Información general del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB

Número de filas duplicadas: 0

Número de valores nulos por columna:
calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

Estadísticas básicas del dataset:
             calls      minutes     messages       mb_used     is_ultra
count  3214.000000  3214.000000  3214.000000   3214.000000  3214.000000
mean     63.038892   438.208787    38.281269  17207.673836     0.306472
std      33.236368   234.569872    36.148326   7570.968246     0.461100
min       0.000000     0.000000     0.000000      0.000000   

<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Buen uso de los métodos de pandas para explorar los datos.
</div>

El dataset contiene 3214 observaciones y 5 columnas sin valores nulos ni duplicados, indicando que está limpio y listo para el análisis. Las variables (calls, minutes, messages, mb_used) muestran una amplia dispersión, con algunos usuarios que no utilizaron ciertos servicios y otros con valores extremos (hasta 244 llamadas, 1632 minutos o casi 50 GB de internet). Además, solo el 30.6% de los usuarios están en el plan Ultra (is_ultra), lo que refleja un desbalance de clases importante para el modelo.

Ahora que ya tenemos la seguridad que los datos están listos para ser procesados se va a proseguir con la siguiente parte. 

Se dividen los datos en tres partes: entrenamiento (60%), validación (20%) y prueba (20%) para asegurar que el modelo aprenda bien y pueda hacer buenas predicciones con datos nuevos. Usamos los datos de entrenamiento para que el modelo aprenda, los de validación para probar diferentes configuraciones y elegir la mejor opción, y los de prueba para ver cómo funcionará en situaciones completamente nuevas. Este método es común porque equilibra bien los datos entre aprender, ajustar y evaluar, y usamos un número fijo (random_state) para que los resultados sean consistentes al repetir el proceso. Además, dividimos los datos al azar para que el análisis sea justo.

In [7]:
# Segmentación del dataset en conjuntos de entrenamiento, validación y prueba
df_train, df_temp = train_test_split(data, test_size=0.4, random_state=54321)  # 60% entrenamiento
df_valid, df_test = train_test_split(df_temp, test_size=0.5, random_state=54321)  # 20% validación y 20% prueba

# Verificar las dimensiones de los conjuntos
print(f"Conjunto de entrenamiento: {df_train.shape}")
print(f"Conjunto de validación: {df_valid.shape}")
print(f"Conjunto de prueba: {df_test.shape}")

# Separar características y variable objetivo del conjunto de entrenamiento
features_train = df_train.drop('is_ultra', axis=1)
target_train = df_train['is_ultra']

# Separar características y variable objetivo del conjunto de validación
features_valid = df_valid.drop('is_ultra', axis=1)
target_valid = df_valid['is_ultra']

Conjunto de entrenamiento: (1928, 5)
Conjunto de validación: (643, 5)
Conjunto de prueba: (643, 5)


<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Se dividieron adecuadamente los datos en entrenamiento y prueba y la variable objetivo se filtro correctamente.
</div>

Con esto comprobamos que los conjuntos están divididos de manera adecuada. 

Ahora se prosigue con el entrenamiento de los modelos. Se entrenarán tres modelos: un árbol de decisión, un bosque aleatorio y una regresión logística, utilizando las características (calls, minutes, messages, mb_used) como variables predictoras y is_ultra como variable objetivo. A continuación, se evaluará el desempeño inicial de cada modelo en el conjunto de validación mediante el cálculo de la exactitud. Posteriormente, se procederá a optimizar los hiperparámetros clave de cada modelo, con el objetivo de mejorar su rendimiento.

**Metodología y justificación de hiperparámetros de árbol de decisión**

Los valores seleccionados para los hiperparámetros permiten realizar un barrido exhaustivo y equilibrado, optimizando el rendimiento del árbol de decisión. El rango de max_depth entre 1 y 20 incluye tanto árboles simples, que evitan el sobreajuste, como árboles más complejos para capturar relaciones detalladas en los datos. Para min_samples_split, se consideraron valores entre 2 y 10, garantizando que las divisiones de los nodos solo se realicen con un mínimo razonable de observaciones, lo que previene particiones innecesarias. Asimismo, min_samples_leaf se probó entre 1 y 10 para evitar que los nodos hoja sean demasiado pequeños, mejorando la estabilidad de las predicciones. Finalmente, los criterios gini y entropy fueron incluidos por ser las métricas estándar en la evaluación de la calidad de las divisiones. Esta configuración permite explorar desde modelos simples hasta configuraciones más complejas, asegurando una optimización adecuada del modelo sin sobrecargar el proceso computacional.

In [12]:
# Valores de hiperparámetros a explorar
max_depths = range (1,21)
min_samples_splits = range (2,11)
min_samples_leaves = range (1,11)
criterions = ['gini', 'entropy']

# Variables para almacenar el mejor modelo y resultados
best_accuracy = 0
best_params = None

# Barrido de hiperparámetros
for depth in max_depths:
    for split in min_samples_splits:
        for leaf in min_samples_leaves:
            for crit in criterions:
                # Crear y entrenar el modelo
                model = DecisionTreeClassifier(
                    max_depth=depth,
                    min_samples_split=split,
                    min_samples_leaf=leaf,
                    criterion=crit,
                    random_state=54321
                )
                model.fit(features_train, target_train)
                predictions = model.predict(features_valid)
                accuracy = accuracy_score(target_valid, predictions)
                
                # Actualizar el mejor modelo
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_params = {
                        'max_depth': depth,
                        'min_samples_split': split,
                        'min_samples_leaf': leaf,
                        'criterion': crit
                    }

# Resultados del mejor modelo
print(f"Mejor exactitud: {best_accuracy:.4f}")
print("Mejores hiperparámetros:")
print(best_params)


Mejor exactitud: 0.7932
Mejores hiperparámetros:
{'max_depth': 7, 'min_samples_split': 2, 'min_samples_leaf': 8, 'criterion': 'entropy'}


<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

¡Muy bien! La exploración de los hiperparámetros se realizó correctamente.
</div>

El modelo de árbol de decisión alcanzó una exactitud óptima del 79.32% tras ajustar los hiperparámetros. Con una profundidad máxima de 7, el árbol logra un equilibrio adecuado entre simplicidad y capacidad para capturar patrones. El mínimo de muestras para dividir fue establecido en 2, permitiendo realizar divisiones siempre que sea posible. Además, el mínimo de muestras en una hoja se fijó en 8, asegurando que cada nodo final tenga suficientes datos para generar predicciones consistentes y robustas. Por último, el criterio de división elegido, entropy, utiliza la ganancia de información como métrica, lo que parece ajustarse mejor a las características del dataset.

Con los valores establecidos se procede a evaluar con el conjunto de prueba y determinar la efectividad del modelo.

In [16]:

# Entrenar el modelo con los mejores hiperparámetros
final_tree_model = DecisionTreeClassifier(
    max_depth=7,
    min_samples_split=2,
    min_samples_leaf=8,
    criterion='entropy',
    random_state=54321
)
final_tree_model.fit(features_train, target_train)

# Evaluar en el conjunto de prueba
test_predictions = final_tree_model.predict(df_test.drop('is_ultra', axis=1))
test_accuracy = accuracy_score(df_test['is_ultra'], test_predictions)

# Mostrar resultados
print(f"Exactitud en el conjunto de prueba: {test_accuracy:.4f}")


Exactitud en el conjunto de prueba: 0.8180


<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Se entrenó correctamente el modelo de árbol de decisión con los mejores hiperparámetros que se obtuvieron.
</div>

**Conclusión Parcial: Árbol de Decisión**

El modelo de árbol de decisión optimizado alcanzó una **exactitud del 81.80%** al ser evaluado en el conjunto de prueba, superando el umbral esperado de 75%. Este resultado indica que el modelo tiene una buena capacidad para generalizar a datos nuevos, logrando un equilibrio adecuado entre simplicidad y precisión gracias a la configuración de hiperparámetros como una profundidad máxima de 7 y un mínimo de 8 muestras en las hojas. Estos ajustes permitieron evitar tanto el sobreajuste como el subajuste, consolidándose como una solución efectiva para clasificar los planes **Smart** y **Ultra**.


<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Buenas conclusiones parciales.
</div>

**Metodología y Justificación de Hiperparámetros para el Bosque Aleatorio**

El modelo de bosque aleatorio, al combinar múltiples árboles de decisión entrenados de forma independiente, requiere la optimización de varios hiperparámetros para lograr un equilibrio entre precisión y eficiencia. El número de árboles (`n_estimators`) se probará con valores de 50, 100 y 150, buscando identificar cómo la estabilidad del modelo mejora con más estimadores, sin incrementar innecesariamente el tiempo de entrenamiento. La profundidad máxima de los árboles (`max_depth`) se evaluará en 5, 10, 15 y sin límite (`None`), permitiendo explorar configuraciones que capturen patrones importantes sin caer en el sobreajuste. El mínimo de muestras necesarias para dividir un nodo (`min_samples_split`) se probará con valores de 2, 5 y 10, representando configuraciones flexibles y conservadoras para las divisiones. Por otro lado, el mínimo de muestras en una hoja (`min_samples_leaf`) se ajustará con valores de 1, 2 y 4, lo que garantiza nodos más robustos sin comprometer la flexibilidad del modelo. Finalmente, se considerarán ambos criterios de división, `gini` y `entropy`, para evaluar cuál optimiza mejor las divisiones en función de las características del dataset. Este barrido, aunque menos exhaustivo, abarca un rango amplio y estratégico que permite encontrar configuraciones óptimas en menor tiempo, mejorando la generalización y precisión del modelo.



In [21]:
# Valores de hiperparámetros ajustados para un barrido más rápido
n_estimators = [50, 100, 150]  
max_depths = [5, 10, 15, None]  
min_samples_splits = [2, 5, 10]  
min_samples_leaves = [1, 2, 4]  
criterions = ['gini', 'entropy'] 

# Variables para almacenar el mejor modelo y resultados
best_accuracy = 0
best_params = None

# Barrido optimizado de hiperparámetros
for n_tree in n_estimators:
    for depth in max_depths:
        for split in min_samples_splits:
            for leaf in min_samples_leaves:
                for crit in criterions:
                    # Crear y entrenar el modelo
                    model = RandomForestClassifier(
                        n_estimators=n_tree,
                        max_depth=depth,
                        min_samples_split=split,
                        min_samples_leaf=leaf,
                        criterion=crit,
                        random_state=54321
                    )
                    model.fit(features_train, target_train)
                    predictions = model.predict(features_valid)
                    accuracy = accuracy_score(target_valid, predictions)
                    
                    # Actualizar el mejor modelo
                    if accuracy > best_accuracy:
                        best_accuracy = accuracy
                        best_params = {
                            'n_estimators': n_tree,
                            'max_depth': depth,
                            'min_samples_split': split,
                            'min_samples_leaf': leaf,
                            'criterion': crit
                        }

# Resultados del mejor modelo
print(f"Mejor exactitud: {best_accuracy:.4f}")
print("Mejores hiperparámetros:")
print(best_params)


Mejor exactitud: 0.7978
Mejores hiperparámetros:
{'n_estimators': 100, 'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 1, 'criterion': 'gini'}


<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Buen uso de los ciclos for anidados para encontrar los mejores hiperparámetros. Ayuda a visualizar mejor lo que se está evaluando.
</div>

<div class="alert alert-block alert-warning">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Este proceso para evaluar distintos hiperparámetros es muy bueno, pero para realizar evaluación de distintos hiperparámetros ya existen librerías que exploran los hiperparámetros como GridSearchCV.
</div>

**Metodología y Justificación de Hiperparámetros para el Bosque Aleatorio**

El modelo de bosque aleatorio alcanzó una **exactitud del 79.78%** tras optimizar sus hiperparámetros. Se seleccionaron los valores óptimos: `n_estimators = 100`, `max_depth = 10`, `min_samples_split = 2`, `min_samples_leaf = 1` y `criterion = 'gini'`. Uno de los principales retos durante el análisis fue el tiempo requerido para ejecutar un barrido exhaustivo, lo que llevó a la decisión de reducir el rango de valores a un conjunto estratégico, equilibrando la profundidad del análisis y la eficiencia computacional. Este enfoque permitió identificar una configuración robusta en un tiempo razonable, asegurando un buen equilibrio entre precisión y estabilidad del modelo.

Ahora se hace un análisis con los datos del conjunto de prueba.


In [22]:
# Entrenar el modelo final con los mejores hiperparámetros
final_forest_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=2,
    min_samples_leaf=1,
    criterion='gini',
    random_state=54321
)
final_forest_model.fit(features_train, target_train)

# Evaluar en el conjunto de prueba
test_predictions = final_forest_model.predict(df_test.drop('is_ultra', axis=1))
test_accuracy = accuracy_score(df_test['is_ultra'], test_predictions)

# Mostrar resultados
print(f"Exactitud en el conjunto de prueba: {test_accuracy:.4f}")


Exactitud en el conjunto de prueba: 0.8336


<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Bien, la exactitud del mejor modelo de bosques aleatorios es muy buena, superaste la solicitada de 0.75
</div>

**Conclusión Parcial: Bosque Aleatorio**

El modelo de bosque aleatorio optimizado alcanzó una **exactitud del 83.36%** en el conjunto de prueba, superando tanto el umbral esperado de 75% como el desempeño del modelo de árbol de decisión. Este resultado demuestra que la estrategia de combinar múltiples árboles de decisión permitió mejorar significativamente la capacidad de generalización del modelo, al reducir el riesgo de sobreajuste y capturar patrones más complejos en los datos. Los hiperparámetros óptimos, como un número moderado de árboles (`n_estimators = 100`), una profundidad máxima controlada (`max_depth = 10`), y el criterio `gini`, contribuyeron a equilibrar precisión y eficiencia. La decisión de limitar el barrido de hiperparámetros debido al tiempo requerido para el análisis resultó ser acertada, permitiendo encontrar una configuración robusta en un tiempo razonable.


**Metodología y Justificación de Hiperparámetros para la Regresión Logística**

El modelo de **regresión logística** se optimizó mediante un barrido de hiperparámetros enfocado en los más relevantes para su desempeño. Se seleccionó el solver `liblinear` debido a su compatibilidad con datasets pequeños y su capacidad para manejar problemas de clasificación binaria de manera eficiente. El hiperparámetro `C`, que controla la fuerza de la regularización, se probó con valores `[0.01, 0.1, 1, 10, 100]`. Este rango permitió explorar desde una regularización fuerte (con valores bajos) hasta una casi inexistente (con valores altos), buscando el mejor equilibrio entre ajuste y generalización. Además, el número máximo de iteraciones (`max_iter`) se fijó en 1000 para garantizar la convergencia del modelo, y se utilizó un `random_state` de 54321 para asegurar la reproducibilidad de los resultados. Este enfoque permitió identificar la configuración óptima para maximizar la exactitud en el conjunto de validación.


In [24]:
# Valores de hiperparámetros a explorar
solvers = ['liblinear']
regularizations = [0.01, 0.1, 1, 10, 100]

# Variables para almacenar el mejor modelo y resultados
best_accuracy = 0
best_params = None

# Barrido de hiperparámetros
for solver in solvers:
    for c in regularizations:
        # Crear y entrenar el modelo
        model = LogisticRegression(
            solver=solver,
            C=c,
            max_iter=1000,
            random_state=54321
        )
        model.fit(features_train, target_train)
        predictions = model.predict(features_valid)
        accuracy = accuracy_score(target_valid, predictions)
        
        # Actualizar el mejor modelo
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = {
                'solver': solver,
                'C': c
            }

# Resultados del mejor modelo
print(f"Mejor exactitud: {best_accuracy:.4f}")
print("Mejores hiperparámetros:")
print(best_params)


Mejor exactitud: 0.6781
Mejores hiperparámetros:
{'solver': 'liblinear', 'C': 0.1}


<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Buen trabajo, la regresión logística se evaluó correctamente, aunque no obtuvo un resultado en exactitud tan sobresaliente, pero esto es por la naturaleza del modelo.
</div>

**Conclusión Parcial: Regresión Logística**

El modelo de regresión logística alcanzó una **exactitud del 67.81%** en el conjunto de validación, siendo inferior al desempeño observado en los modelos de árbol de decisión y bosque aleatorio. Esta diferencia puede atribuirse a las limitaciones de la regresión logística, ya que asume relaciones lineales entre las características y la variable objetivo, lo que la hace menos efectiva en datasets con patrones más complejos o no lineales. Aunque podrían explorarse mejoras como el escalado de las características, la creación de nuevas variables o el ajuste de parámetros adicionales, estas opciones no se consideraron debido a restricciones dentro del sprint impartido. 


**Prueba de Cordura**

La prueba de cordura se realizó para verificar que el modelo optimizado funciona significativamente mejor que un modelo aleatorio, garantizando que los resultados no son producto del azar. Para ello, se creó un modelo aleatorio que asigna etiquetas de manera aleatoria a los datos del conjunto de prueba, respetando la proporción de las clases presentes en el dataset. Posteriormente, se comparó la exactitud del modelo aleatorio con la del modelo optimizado.

El modelo con el mejor desempeño fue el **bosque aleatorio**, que alcanzó una **exactitud del 83.36%** en el conjunto de prueba. Este modelo utilizó los siguientes hiperparámetros óptimos:
- **Número de árboles (`n_estimators`)**: 100.
- **Profundidad máxima (`max_depth`)**: 10.
- **Mínimo de muestras para dividir (`min_samples_split`)**: 2.
- **Mínimo de muestras en una hoja (`min_samples_leaf`)**: 1.
- **Criterio de división (`criterion`)**: `gini`.

Al comparar este modelo con el desempeño de un modelo aleatorio, se espera que el bosque aleatorio supere ampliamente al modelo aleatorio, demostrando que los resultados son significativos y no aleatorios.


In [26]:
# Generar predicciones aleatorias basadas en la proporción de clases en el conjunto de prueba
random_predictions = np.random.choice(
    df_test['is_ultra'].unique(),  # Clases posibles (0 y 1)
    size=len(df_test),             # Número de predicciones a generar
    p=df_test['is_ultra'].value_counts(normalize=True).values  # Probabilidades basadas en la distribución de clases
)

# Calcular la exactitud del modelo aleatorio
random_accuracy = accuracy_score(df_test['is_ultra'], random_predictions)

# Mostrar resultados
print(f"Exactitud del modelo aleatorio: {random_accuracy:.4f}")
print(f"Exactitud del mejor modelo (Bosque Aleatorio): 0.8336")  # Cambiar si el mejor modelo cambia

# Verificar si el modelo optimizado supera la prueba de cordura
if random_accuracy < 0.8336:  # Sustituir por la exactitud del mejor modelo si es diferente
    print("El modelo optimizado supera la prueba de cordura.")
else:
    print("El modelo optimizado no supera la prueba de cordura.")


Exactitud del modelo aleatorio: 0.6470
Exactitud del mejor modelo (Bosque Aleatorio): 0.8336
El modelo optimizado supera la prueba de cordura.


<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Buen trabajo, la prueba de cordura se realizó correctamente.
</div>

**Prueba de Cordura: Resultados**

El modelo aleatorio alcanzó una **exactitud del 64.70%**, consistente con la distribución de clases en el conjunto de prueba. En comparación, el modelo optimizado de **bosque aleatorio** logró una **exactitud del 83.36%**, superando significativamente al modelo aleatorio. Este resultado confirma que el modelo optimizado tiene un desempeño superior y que no clasifica los datos de forma aleatoria, sino que identifica patrones relevantes que mejoran la predicción de los planes **Smart** y **Ultra**. Por lo tanto, el modelo optimizado supera satisfactoriamente la prueba de cordura.


**Conclusión General del Proyecto**

El proyecto permitió desarrollar y evaluar diferentes modelos de machine learning para clasificar los planes **Smart** y **Ultra** de la compañía Megaline, logrando resultados satisfactorios. Se probaron tres modelos principales: árbol de decisión, bosque aleatorio y regresión logística. El **bosque aleatorio** optimizado demostró ser el modelo más efectivo, alcanzando una **exactitud del 83.36%** en el conjunto de prueba, superando ampliamente el umbral esperado del 75% y la exactitud del modelo aleatorio (64.70%). Este resultado valida su capacidad para capturar patrones significativos en los datos, mientras mantiene un equilibrio entre precisión y eficiencia computacional. Aunque la regresión logística tuvo un desempeño limitado (67.81%), fue útil como referencia para entender las limitaciones de modelos lineales en problemas más complejos. En conclusión, el bosque aleatorio no solo demostró ser robusto y fiable, sino que superó la prueba de cordura, consolidándose como la mejor opción para recomendar los planes tarifarios basándose en los datos de comportamiento de los usuarios.


<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Buenas conclusiones, se resume lo realizado a lo largo del proyecto.
</div>

<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

¡Hola! 

Te felicito por tu proyecto, está muy bien realizado y completo, exploraste varios modelos y en cada modelo usaste una muy buena técnica de exploración de sus hiperparámetros. Obtuviste un buen nivel de exactitud de 83%, superando el umbral de 75% y de la prueba de cordura. Bien hecho, puedo aprobar tu proyecto, éxito en los siguientes sprints.
    
Saludos, Marcos.
</div>